# **📓 03: Lanzador de la Interfaz Web con Streamlit**

Este notebook tiene una única y clara responsabilidad: lanzar la interfaz de usuario web interactiva para el proyecto VIREC. La aplicación reside en su propio directorio (`streamlit_app/`) y este notebook actúa como el centro de control para seleccionarla y ejecutarla en un entorno de Colab.

**Flujo de Trabajo:**
1.  **Instalación y Configuración:** Se instalan las librerías, se monta Google Drive y se localiza dinámicamente la carpeta del proyecto.
2.  **Selección de Modelo:** El script lee el registro central de experimentos (`registro_de_experimentos.csv`) y te permite seleccionar interactivamente qué modelo deseas cargar en la interfaz a través de un menú desplegable.
3.  **Lanzamiento:** El script ejecuta la aplicación `streamlit_app/app.py`, pasándole la ruta al modelo seleccionado y sus parámetros como argumentos de línea de comandos. Luego, crea un túnel público con `ngrok` para hacer la aplicación accesible desde cualquier navegador.

### **Configuración y Selección de Modelo**
Esta celda se encarga de toda la preparación. Conecta Google Drive, localiza el proyecto y lee el `registro_de_experimentos.csv` para encontrar todos los modelos que has entrenado.

👇 **Por favor, selecciona del menú desplegable el experimento que deseas cargar en la interfaz web antes de continuar con la siguiente celda.**

In [ ]:
# ====================================================================================
# @title PASO 1: SETUP Y SELECCIÓN DE MODELO A LANZAR
# ====================================================================================

# --- Instalación de Librerías ---
!pip install -r ../requirements.txt -q

from google.colab import drive
import os
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# --- Montar Drive y buscar la ruta del proyecto ---
if not os.path.exists('/content/drive/MyDrive'): drive.mount('/content/drive')
NOMBRE_CARPETA_ANCLA = 'Proyecto_VIREC'
RUTA_BASE_PROYECTO = None
for root, dirs, files in os.walk('/content/drive/MyDrive'):
    if NOMBRE_CARPETA_ANCLA in dirs:
        RUTA_BASE_PROYECTO = os.path.join(root, NOMBRE_CARPETA_ANCLA)
        break
if not RUTA_BASE_PROYECTO: raise FileNotFoundError(f"No se encontró la carpeta '{NOMBRE_CARPETA_ANCLA}'.")
print(f"✅ Proyecto encontrado en: {RUTA_BASE_PROYECTO}")


# --- Leer el registro de experimentos para poblar el dropdown ---
ruta_registro = os.path.join(RUTA_BASE_PROYECTO, 'modelos_entrenados', 'registro_de_experimentos.csv')
modelos_info = {} # Diccionario para guardar toda la info del modelo

if not os.path.exists(ruta_registro):
    print(f"❌ ADVERTENCIA: No se encontró el archivo de registro.")
else:
    df_registro = pd.read_csv(ruta_registro)
    for index, row in df_registro.iterrows():
        display_name = f"{row['experiment_id']} (val_acc: {row['best_val_accuracy']})"
        # Guardamos la ruta completa y el tamaño de la imagen
        modelos_info[display_name] = {
            "ruta_completa": os.path.join(RUTA_BASE_PROYECTO, row['ruta_modelo_guardado']),
            "img_size": int(row['img_size'])
        }
    
    dropdown_modelos = widgets.Dropdown(
        options=modelos_info.keys(),
        description='Elige un experimento:',
        style={'description_width': 'initial'}, layout={'width': 'max-content'}
    )
    print("\n👇 Por favor, selecciona el experimento a lanzar:")
    display(dropdown_modelos)

### **Lanzamiento de la Aplicación Web**
Esta celda final toma tu selección del menú anterior y lanza la aplicación de Streamlit.

**Proceso de Ejecución:**
1.  Obtiene la ruta completa al modelo `.keras` y el `img_size` correspondiente a tu selección.
2.  Limpia cualquier sesión anterior de Streamlit o ngrok.
3.  Ejecuta el comando `streamlit run app.py`, pasándole la ruta y el tamaño de la imagen como argumentos.
4.  Crea un túnel público con ngrok.

**Importante:** Antes de ejecutar, asegúrate de haber creado el archivo `config.py` en la raíz de tu proyecto a partir de la plantilla y de haber añadido tu token de Ngrok.

In [ ]:
# ====================================================================================
# @title PASO 2: LANZAR LA APLICACIÓN WEB
# ====================================================================================
from pyngrok import ngrok
import time
import sys

# --- Importar la configuración de NGROK ---
sys.path.append(RUTA_BASE_PROYECTO)
try:
    from config import NGROK_AUTH_TOKEN
except ImportError:
    NGROK_AUTH_TOKEN = None

if NGROK_AUTH_TOKEN and 'dropdown_modelos' in locals():
    # --- 1. Obtener los parámetros del modelo seleccionado ---
    seleccion = dropdown_modelos.value
    info_modelo = modelos_info[seleccion]
    ruta_modelo_a_lanzar = info_modelo['ruta_completa']
    img_size_a_lanzar = info_modelo['img_size']
    
    print(f"🚀 Preparando para lanzar el modelo: {os.path.basename(ruta_modelo_a_lanzar)}")
    print(f"   Con tamaño de imagen: {img_size_a_lanzar}x{img_size_a_lanzar}")

    # --- 2. Limpiar sesiones anteriores ---
    os.system("killall streamlit")
    ngrok.kill()
    
    # --- 3. Moverse al directorio de la app ---
    RUTA_APP = os.path.join(RUTA_BASE_PROYECTO, 'streamlit_app')
    %cd {RUTA_APP}

    # --- 4. Lanzar la app con los argumentos correctos ---
    # Usamos f-string para construir el comando
    # El '--' es crucial para decirle a streamlit que lo que sigue son argumentos para app.py
    comando_lanzamiento = f"nohup streamlit run app.py -- -- '{ruta_modelo_a_lanzar}' {img_size_a_lanzar} &"
    get_ipython().system(comando_lanzamiento)
    
    time.sleep(5)
    
    # --- 5. Conectar ngrok ---
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)
    public_url = ngrok.connect(8501)
    print("\n" + "="*60)
    print("🎉 ¡Tu aplicación está en línea!")
    print(f"🔗 Abre esta URL en tu navegador: {public_url}")
    print("="*60)
    
    %cd /content
else:
    print("❌ ERROR: Asegúrate de ejecutar la celda anterior y de tener tu token de Ngrok en config.py.")